In [1]:
import os
os.chdir("../")
%pwd

'/Users/valentinmonney/Documents/data_science/python/MachineLearningProjects/Kidney-Disease-Classification-Deep-Learning-Project'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    """Configuration class for preparing base model."""

    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [3]:
from cnn_classifier.constants import *
from cnn_classifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    """Class for managing configuration settings."""

    def __init__(
        self,
        config_filepath: str = CONFIG_FILE_PATH,
        params_filepath: str = PARAMS_FILE_PATH,
    ) -> None:
        """Initialize the ConfigurationManager class.

        Args:
        ----
            config_filepath: The path to the configuration file.
            params_filepath: The path to the parameters file.
        """
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """Return the prepare base model configuration."""
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        return PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
        )

In [1]:
import tensorflow as tf

print(tf.__version__)

2023-10-18 17:34:10.610926: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.12.1


In [6]:
class PrepareBaseModel:
    """Prepare the base model for training."""

    def __init__(self, config: PrepareBaseModelConfig) -> None:
        """Initialize the class."""
        self.config = config

    def get_base_model(self) -> None:
        """Retrieve the base VGG 16 model and save it."""
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            include_top=self.config.params_include_top,
            weights=self.config.params_weights,
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(
            flatten_in
        )

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"],
        )

        full_model.summary()
        return full_model

    def update_base_model(self) -> None:
        """Update the base model with the new number of classes."""
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate,
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model) -> None:
        """Save the model."""
        model.save(path)

AttributeError: module 'keras.api._v2.keras' has no attribute 'Model'

In [ ]:
from cnn_classifier import logger

STAGE_NAME = "Prepare base model"


class PrepareBaseModelTrainingPipeline:
    """Prepare base model training pipeline."""

    def __init__(self) -> None:
        """Initialize the PrepareBaseModelTrainingPipeline class."""

    def main(self) -> None:
        """Run the data ingestion pipeline."""
        config = ConfigurationManager()
        prepare_base_model_config = config.get_prepare_base_model_config()
        prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
        prepare_base_model.get_base_model()
        prepare_base_model.update_base_model()


try:
    logger.info("*******************")
    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
    obj = PrepareBaseModelTrainingPipeline()
    obj.main()
    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
    logger.exception(e)
    raise

[2023-10-18 16:16:00,937: INFO: 2261882295] *******************
[2023-10-18 16:16:00,940: INFO: 2261882295] >>>>>> stage Prepare base model started <<<<<<
[2023-10-18 16:16:00,947: INFO: common] yaml file: config/config.yaml loaded successfully
[2023-10-18 16:16:00,952: INFO: common] yaml file: params.yaml loaded successfully
[2023-10-18 16:16:00,953: INFO: common] created directory at: artifacts
[2023-10-18 16:16:00,955: INFO: common] created directory at: artifacts/prepare_base_model
[2023-10-18 16:16:01,372: WARNING: saving_utils] Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (